# Exercises Discriminant Analysis

## Theoretical questions

### Question 1:
- Zoek in de Canvas-cursus de dataset `bordeaux` en plaats die in een dataframe.
    - Pas een discriminantanalyse toe op deze data set waarbij quality als afhankelijke variabele optreedt en temperature, sun, heat en rain als de onafhankelijke variabelen.
    - Hoeveel dimensies heeft de discriminantanalyse?
    - Maak een plot van de resultaten van de discriminantanalyse. Wat wordt er hier gevisualiseerd?
    - Pas het predict commando even toe op de oorspronkelijke gegevens. Hoeveel percent van de wijnen krijgen dezelfde label opgeplakt via de discriminantanalyse als geobserveerd?

### Question 2:
- Zoek in de Canvas-cursus de dataset `birthwt` en plaats die in een dataframe.
    - Analyseer het type van gegevens in deze dataset.
    - Welke kolommen kunnen er als afhankelijke variabele voor een discriminant analyse worden gebruikt?
    - Welke kolommen kunnen er als onafhankelijke variabelen voor een discriminant analyse worden gebruikt?
    - Pas een discriminantanalyse toe op deze data set (maakt gebruik van je keuzes hierboven).
    - Hoeveel dimensies heeft de discriminantanalyse?
    - Maak een plot van de resultaten van de discriminantanalyse.



### Question 3:
- Zoek op Canvas de dataset `Cars93` en plaats die in een dataframe.
    - Analyseer de type van gegevens in deze dataset.
    - Splits de data set in twee: de eerste 90 rijen als ‘leer data set’ waarom je een discriminant analyse gaat toepassen (zie e) en de laatste 3 rijen om na te gaan of de bekomen discriminant analyse deze auto’s in de juiste klasse plaatst (zie g).
    - Welke kolommen kunnen er als afhankelijke variabele voor een discriminant analyse worden gebruikt?
    - Welke kolommen kunnen er als onafhankelijke variabelen voor een discriminant analyse worden gebruikt?
    - Pas een discriminantanalyse toe op deze data set. Neem als afhankelijke variabele de kolom Type en als onafhankelijke variabelen alle bruikbare kolommen (zie 4).
    - Hoeveel dimensies heeft de discriminantanalyse?
    - Pas de bekomen discriminantanalyse met behulp van predict-commando toe op de laatste 3 rijen van de oorspronkelijke gegevens (zie b). Worden alle 3 de wagens in de juiste klasse geplaatst?